<a href="https://colab.research.google.com/github/ysurs/cuda-100-days/blob/main/cuda_day1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile vector_add.cu

#include <stdio.h>
__global__ void vecAddKernel(int *A, int *B, int *C, int n) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < n) {
        C[idx] = A[idx] + B[idx];
    }
}

int main() {
    const int n = 5;
    int a[n], b[n], c[n];
    int size = n * sizeof(int);

    // This will store address of allocated space on device
    int *a_d,*b_d,*c_d;

    cudaMalloc((void **)&a_d,size);
    cudaMalloc((void **)&b_d,size);
    cudaMalloc((void **)&c_d,size);

    for (int i = 0; i < n; ++i) {
        a[i] = b[i] = i;
        printf("a[%d] = %d\n", i, a[i]);
        printf("b[%d] = %d\n", i, b[i]);
    }


    cudaMemcpy(a_d,a,size,cudaMemcpyHostToDevice);
    cudaMemcpy(b_d,b,size,cudaMemcpyHostToDevice);

    vecAddKernel<<<ceil(n/256.0), 256>>>(a_d, b_d, c_d, n);

    cudaMemcpy(c,c_d,size,cudaMemcpyDeviceToHost);


    for (int i = 0; i < 5; ++i) {
        printf("c[%d] = %d\n", i, c[i]);
    }

    cudaFree(a_d);
    cudaFree(b_d);
    cudaFree(c_d);
}

Overwriting vector_add.cu


In [5]:
!nvcc vector_add.cu -o vector_add

In [6]:
!./vector_add

a[0] = 0
b[0] = 0
a[1] = 1
b[1] = 1
a[2] = 2
b[2] = 2
a[3] = 3
b[3] = 3
a[4] = 4
b[4] = 4
c[0] = 0
c[1] = 2
c[2] = 4
c[3] = 6
c[4] = 8
